In [1]:
import pandas as pd
import spacy

In [27]:
df = pd.read_csv('twitter_training.csv')
df.columns = ['drop', 'brand', 'sentiment', 'doc']
df.drop('drop', axis=1, inplace=True)
df.dropna(inplace=True)

df.head()

,brand,sentiment,doc
0,Borderlands,Positive,I am coming to the borders and I will kill you...
1,Borderlands,Positive,im getting on borderlands and i will kill you ...
2,Borderlands,Positive,im coming on borderlands and i will murder you...
3,Borderlands,Positive,im getting on borderlands 2 and i will murder ...
4,Borderlands,Positive,im getting into borderlands and i can murder y...


In [28]:
df.isna().sum()

brand        0
sentiment    0
doc          0
dtype: int64

In [29]:
df['sentiment'].value_counts() / df.shape[0]

sentiment
Negative      0.302156
Positive      0.279127
Neutral       0.244719
Irrelevant    0.173998
Name: count, dtype: float64

In [30]:
df = df.sample(1000).copy()

In [34]:
df['sentiment'].value_counts() / df.shape[0]

sentiment
Negative      0.311
Positive      0.267
Neutral       0.241
Irrelevant    0.181
Name: count, dtype: float64

In [35]:
import en_core_web_sm

nlp = en_core_web_sm.load()

In [37]:
df['spacy'] = df['doc'].apply(nlp)

In [39]:
df.to_csv('temp.csv', index=False)

In [41]:
import os 
file = os.listdir('../02Dbscan')[0]

['customers.csv',
 '02-OPTIONAL_Lets_buy_groceries.ipynb',
 'DemoCodeAlong.ipynb',
 '02-Code_DBSCAN.ipynb',
 '.venv',
 '01-911_emergencies.ipynb',
 '911.csv']

In [43]:
type(pd.read_csv('temp.csv')['spacy'].loc[0])

str

In [62]:
from sklearn.feature_extraction.text import CountVectorizer
from spacy.lang.en.stop_words import STOP_WORDS

cv = CountVectorizer(stop_words=list(STOP_WORDS))
X = cv.fit_transform(df['doc'])

/home/gmanderscheid/jedha/.venv/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ll', 've'] not in stop_words.
  warnings.warn(


In [63]:
crash_my_computer = pd.DataFrame(X.toarray(), columns=cv.get_feature_names_out())

In [68]:
stop = list(STOP_WORDS)
stop.extend(crash_my_computer.sum()[crash_my_computer.sum() < 10].index)

In [55]:
crash_my_computer.sum().sort_values(ascending=False)

the          596
to           364
and          334
of           244
it           240
            ... 
greatly        1
greedy         1
grenade        1
grilled        1
govmurphy      1
Length: 4271, dtype: int64

In [69]:
cv = CountVectorizer(stop_words=stop)
X = cv.fit_transform(df['doc'])

/home/gmanderscheid/jedha/.venv/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ve'] not in stop_words.
  warnings.warn(


In [70]:
crash_my_computer = pd.DataFrame(X.toarray(), columns=cv.get_feature_names_out())


In [71]:
crash_my_computer

,10,100,12,19,20,2020,account,actually,amazing,amazon,...,work,working,world,wtf,xbox,year,years,yesterday,youtu,youtube
0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
997,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [73]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

In [75]:
le = LabelEncoder()
df['sentiment'] = le.fit_transform(df['sentiment'])

In [76]:
X_train, X_test, y_train, y_test = train_test_split(X, df['sentiment'], random_state=0)

In [80]:
rfc = RandomForestClassifier(max_depth=10)
rfc.fit(X_train, y_train)
rfc.score(X_train, y_train), rfc.score(X_test, y_test)

(0.6133333333333333, 0.468)

In [81]:
df['sentiment']

27140    0
68962    1
29601    1
2465     3
47849    0
        ..
2090     1
15669    2
27890    3
3687     0
52060    3
Name: sentiment, Length: 1000, dtype: int64